In [59]:
import numpy as np
import pandas as pd

In [60]:
train = pd.read_csv('yds_train2018.csv')
test = pd.read_csv('yds_test2018.csv')
holiday = pd.read_excel('holidays.xlsx')
expenses = pd.read_csv('promotional_expense.csv')
sample = pd.read_csv('sample_submission.csv')

In [61]:
train.drop(['Merchant_ID', 'Week'], axis=1, inplace=True)

In [62]:
train= train.groupby(['Year', 'Month', 'Product_ID','Country'], as_index=False).agg({"Sales":"sum"})

In [63]:
test_sno = test.S_No
test.drop(['S_No'], axis = 1, inplace=True)

In [64]:
expenses.rename(columns={'Product_Type' : 'Product_ID'}, inplace=True)

In [65]:
combin = pd.concat([train,test])

In [66]:
combin2 = pd.merge(left = combin, right = expenses, how='left')

In [67]:
combin2['Expense_Price'] = combin2.groupby(['Country','Year'])['Expense_Price'].apply(lambda x : x.fillna(x.mean()))

In [68]:
combin2.Year = combin2.Year.astype('category')

In [69]:
combin2.Product_ID = combin2.Product_ID.astype('category')

In [70]:
combin2.Month = combin2.Month.astype('category')

In [71]:
combin2 = pd.get_dummies(combin2, drop_first=True)
#combin2['Expense_Price'] = np.log1p(combin2['Expense_Price'])

In [72]:
train2 = combin2[~combin2.Sales.isnull()]

In [73]:
test2 = combin2[combin2.Sales.isnull()]

In [74]:
corrmat = train2.corr()
train_X = train2.drop('Sales', axis = 1)
train_y = train2['Sales']

In [75]:
import lightgbm as lgb
dtrainlgb = lgb.Dataset(data = train_X, label = np.log1p(train_y))
params = {'objective': 'mape',
         'learning_rate' : 0.1,
          'num_leaves' : 32
         }
cv_results = lgb.cv(params,
                    dtrainlgb,
                    num_boost_round = 1000,
                    nfold = 10,
                    metrics = 'mape',
                    early_stopping_rounds = 10,
                    verbose_eval = 1,
                    stratified = False
                   )         

[1]	cv_agg's mape: 0.102687 + 0.0119094
[2]	cv_agg's mape: 0.0935027 + 0.0106846
[3]	cv_agg's mape: 0.0853682 + 0.00953714
[4]	cv_agg's mape: 0.0779952 + 0.00862447
[5]	cv_agg's mape: 0.0713851 + 0.00763557
[6]	cv_agg's mape: 0.0654943 + 0.00678154
[7]	cv_agg's mape: 0.0602175 + 0.00617478
[8]	cv_agg's mape: 0.05548 + 0.00593402
[9]	cv_agg's mape: 0.0510556 + 0.00561835
[10]	cv_agg's mape: 0.047309 + 0.00547287
[11]	cv_agg's mape: 0.0439919 + 0.00568592
[12]	cv_agg's mape: 0.0408553 + 0.00561787
[13]	cv_agg's mape: 0.0380475 + 0.005592
[14]	cv_agg's mape: 0.0354127 + 0.00552832
[15]	cv_agg's mape: 0.0331952 + 0.00553701
[16]	cv_agg's mape: 0.0310837 + 0.00544418
[17]	cv_agg's mape: 0.0293619 + 0.00540839
[18]	cv_agg's mape: 0.0276976 + 0.00539341
[19]	cv_agg's mape: 0.0260957 + 0.00533122
[20]	cv_agg's mape: 0.0247237 + 0.00531191
[21]	cv_agg's mape: 0.0235317 + 0.00534403
[22]	cv_agg's mape: 0.0224732 + 0.0053346
[23]	cv_agg's mape: 0.0215096 + 0.00530297
[24]	cv_agg's mape: 0.0206539

In [76]:
lgbmodel = lgb.train(params,
                    dtrainlgb,
                    num_boost_round = 136)

In [77]:
# import matplotlib.pyplot as plt
# a = lgb.plot_importance(lgbmodel, max_num_features=20, height=.2, figsize=(12,9))
# plt.show()

In [78]:
lgb_pred = lgbmodel.predict(train_X)

In [79]:
def smape_calc(pred, original):
    err = np.mean(np.abs(pred-original) / ((np.abs(pred+original))/2))
    return(err)    

In [80]:
smape = smape_calc((np.exp(lgb_pred)-1), train_y)
smape

0.1261390711819645

In [23]:
test2.drop('Sales', axis = 1, inplace=True)

/home/tinku/.virtualenvs/cv/lib/python3.5/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [24]:
test_pred = lgbmodel.predict(test2)

In [25]:
test = pd.read_csv('yds_test2018.csv')

In [26]:
test.Sales = np.exp(test_pred)-1

In [27]:
test.Sales = test.Sales.round(0)

In [28]:
test.to_csv('submit22.csv', index = False)

In [29]:
test.head()
#test.Sales.round(0)

,S_No,Year,Month,Product_ID,Country,Sales
0,79073,2016,4,1,Argentina,14538225.0
1,79074,2016,5,1,Argentina,33524718.0
2,79075,2016,6,1,Argentina,14609195.0
3,79076,2016,7,1,Argentina,14821599.0
4,79077,2016,8,1,Argentina,14476930.0
